In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import KFold

2024-04-03 05:52:50.668016: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 05:52:51.922415: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 05:52:55.274397: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 05:52:58.665742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:

# Fungsi untuk membangun model
# def build_model():
#     model = Sequential([
#         Flatten(input_shape=(28, 28)),
#         Dense(128, activation='relu'),
#         Dense(60, activation='softmax')
#     ])
#     return model

# Fungsi
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [6]:
# Membuat dataset menggunakan tf.keras.preprocessing.image_dataset_from_directory
mnist = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)

Found 240 files belonging to 60 classes.


In [7]:
# Preprocessing data
def preprocess_data(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Preprocess data train
mnist = mnist.map(preprocess_data)

In [8]:
# Pemilihan kombinasi batch size dan jumlah epoch yang akan dievaluasi [Kombinasi Belum Fix]
batch_sizes = [30]
num_epochs = [100]

In [11]:
# Inisialisasi k-Fold
kf = KFold(n_splits=2, shuffle=False, random_state=None)

In [12]:
# Mengeksekusi eksperimen untuk setiap kombinasi
results = {}
for batch_size in batch_sizes:
    for num_epoch in num_epochs:
        print(f"Training with batch size {batch_size} and {num_epoch} epochs...")
        val_accuracies = []

        # Mengambil indeks dari dataset
        indices = np.arange(len(mnist))

        for train_index, val_index in kf.split(indices):
            # Mengambil data pelatihan dan validasi menggunakan indeks
            # error out of range
            train_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, train_index))).batch(batch_size)
            val_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, val_index))).batch(batch_size)

            model = build_resnet50_model()
            model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

            # # Melatih model dengan kombinasi yang dipilih
            # history = model.fit(train_data, epochs=num_epoch, verbose=0)

            # Evaluasi model pada data validasi
            # _, val_accuracy = model.evaluate(val_data, verbose=0)
            # val_accuracies.append(val_accuracy)
            
             # Melatih model dengan kombinasi yang dipilih
            history = model.fit(train_data, epochs=num_epoch, verbose=0)

            # Evaluasi model pada data validasi
            val_accuracy = history.history['sparse_categorical_accuracy'][-1]
            val_accuracies.append(val_accuracy)

        # Menyimpan hasil evaluasi
        results[(batch_size, num_epoch)] = np.mean(val_accuracies)
        # results[(batch_size, num_epoch)] = history.history


Training with batch size 30 and 100 epochs...


2024-03-28 02:22:51.874944: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/codespace/.python/current/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-03-28 02:22:52.100998: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:22:52.307187: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:22:52.612386: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is abort

Training with batch size 30 and 1000 epochs...


2024-03-28 02:25:19.351878: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:25:19.648980: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:25:19.859356: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:25:20.076037: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:25:20.297381: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:25:20.495535: W tensorflow/core/framework/local_rendezvous.cc:404]

KeyboardInterrupt: 

In [13]:
# Inisialisasi k-Fold
kf = KFold(n_splits=2, shuffle=True, random_state=123)
# Mengeksekusi eksperimen untuk setiap kombinasi
results = {}
for batch_size in batch_sizes:
    for num_epoch in num_epochs:
        print(f"Training with batch size {batch_size} and {num_epoch} epochs...")
        val_accuracies = []

        # Mengambil indeks dari dataset
        indices = np.arange(len(mnist))

        for train_index, val_index in kf.split(indices):
            # Mengambil data pelatihan dan validasi menggunakan indeks
            # error out of range
            train_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, train_index))).batch(batch_size)
            val_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, val_index))).batch(batch_size)

            model = build_resnet50_model()
            model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

            # # Melatih model dengan kombinasi yang dipilih
            # history = model.fit(train_data, epochs=num_epoch, verbose=0)

            # Evaluasi model pada data validasi
            # _, val_accuracy = model.evaluate(val_data, verbose=0)
            # val_accuracies.append(val_accuracy)
            
             # Melatih model dengan kombinasi yang dipilih
            history = model.fit(train_data, epochs=num_epoch, verbose=0)

            # Evaluasi model pada data validasi
            val_accuracy = history.history['sparse_categorical_accuracy'][-1]
            val_accuracies.append(val_accuracy)

        # Menyimpan hasil evaluasi
        results[(batch_size, num_epoch)] = np.mean(val_accuracies)
        # results[(batch_size, num_epoch)] = history.history


Training with batch size 30 and 100 epochs...


2024-03-28 02:41:30.029095: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:41:30.223843: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:41:30.407800: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:41:30.731244: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:41:30.912851: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:41:31.093809: W tensorflow/core/framework/local_rendezvous.cc:404]

Training with batch size 30 and 1000 epochs...


2024-03-28 02:42:24.609161: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:42:24.922115: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:42:25.121668: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:42:25.298425: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:42:25.473221: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:42:25.779692: W tensorflow/core/framework/local_rendezvous.cc:404]

Training with batch size 32 and 100 epochs...


2024-03-28 02:49:49.947171: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:49:50.144719: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:49:50.345020: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:49:50.636708: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:49:50.830802: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:49:51.022994: W tensorflow/core/framework/local_rendezvous.cc:404]

Training with batch size 32 and 1000 epochs...


2024-03-28 02:50:45.628947: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:50:45.818040: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:50:45.997995: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:50:46.179516: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:50:46.364286: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 02:50:46.548190: W tensorflow/core/framework/local_rendezvous.cc:404]

In [14]:
# Analisis hasil
best_accuracy = 0
best_combination = None
for combination, val_accuracy in results.items():
    # val_accuracy = history['val_sparse_categorical_accuracy'][-1]
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_combination = combination

    print(f"Batch size: {combination[0]}, Epochs: {combination[1]}")
    print("Average Validation Accuracy:", val_accuracy)

print(f"Best combination: batch size {best_combination[0]}, {best_combination[1]} epochs. Validation accuracy: {best_accuracy}")

Batch size: 30, Epochs: 100
Average Validation Accuracy: 1.0
Batch size: 30, Epochs: 1000
Average Validation Accuracy: 1.0
Batch size: 32, Epochs: 100
Average Validation Accuracy: 1.0
Batch size: 32, Epochs: 1000
Average Validation Accuracy: 1.0
Best combination: batch size 30, 100 epochs. Validation accuracy: 1.0


K-FOLD = 5

In [15]:
# Inisialisasi k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
# Mengeksekusi eksperimen untuk setiap kombinasi
results = {}
for batch_size in batch_sizes:
    for num_epoch in num_epochs:
        print(f"Training with batch size {batch_size} and {num_epoch} epochs...")
        val_accuracies = []

        # Mengambil indeks dari dataset
        indices = np.arange(len(mnist))

        for train_index, val_index in kf.split(indices):
            # Mengambil data pelatihan dan validasi menggunakan indeks
            # error out of range
            train_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, train_index))).batch(batch_size)
            val_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, val_index))).batch(batch_size)

            model = build_resnet50_model()
            model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

            # # Melatih model dengan kombinasi yang dipilih
            # history = model.fit(train_data, epochs=num_epoch, verbose=0)

            # Evaluasi model pada data validasi
            # _, val_accuracy = model.evaluate(val_data, verbose=0)
            # val_accuracies.append(val_accuracy)
            
             # Melatih model dengan kombinasi yang dipilih
            history = model.fit(train_data, epochs=num_epoch, verbose=0)

            # Evaluasi model pada data validasi
            val_accuracy = history.history['sparse_categorical_accuracy'][-1]
            val_accuracies.append(val_accuracy)

        # Menyimpan hasil evaluasi
        results[(batch_size, num_epoch)] = np.mean(val_accuracies)
        # results[(batch_size, num_epoch)] = history.history


Training with batch size 30 and 100 epochs...


2024-03-28 03:01:09.435014: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:01:09.738545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:01:09.991175: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:01:10.293361: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:01:10.501720: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:01:10.706194: W tensorflow/core/framework/local_rendezvous.cc:404]

Training with batch size 30 and 1000 epochs...


2024-03-28 03:03:39.558472: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:03:39.783812: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:03:39.994021: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:03:40.294218: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:03:40.599530: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:03:40.831005: W tensorflow/core/framework/local_rendezvous.cc:404]

Training with batch size 32 and 100 epochs...


2024-03-28 03:24:28.111310: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:24:28.428932: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:24:28.655948: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:24:28.953249: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:24:29.268276: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:24:29.485002: W tensorflow/core/framework/local_rendezvous.cc:404]

Training with batch size 32 and 1000 epochs...


2024-03-28 03:26:57.614858: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:26:57.872284: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:26:58.081387: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:26:58.289618: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:26:58.600165: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-28 03:26:58.831238: W tensorflow/core/framework/local_rendezvous.cc:404]

In [16]:
# Analisis hasil
best_accuracy = 0
best_combination = None
for combination, val_accuracy in results.items():
    # val_accuracy = history['val_sparse_categorical_accuracy'][-1]
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_combination = combination

    print(f"Batch size: {combination[0]}, Epochs: {combination[1]}")
    print("Average Validation Accuracy:", val_accuracy)

print(f"Best combination: batch size {best_combination[0]}, {best_combination[1]} epochs. Validation accuracy: {best_accuracy}")

Batch size: 30, Epochs: 100
Average Validation Accuracy: 1.0
Batch size: 30, Epochs: 1000
Average Validation Accuracy: 1.0
Batch size: 32, Epochs: 100
Average Validation Accuracy: 1.0
Batch size: 32, Epochs: 1000
Average Validation Accuracy: 1.0
Best combination: batch size 30, 100 epochs. Validation accuracy: 1.0
